In [3]:
# MEK Import block
import astropy
from astropy.io import fits
import numpy as np
import os
from astropy.coordinates import SkyCoord
import astropy.units as u
import scipy
import math
from sdss_semaphore.targeting import TargetingFlags

In [2]:
# MEK grab data from fits file
dr16file = fits.open("../../DR16Q_v4.fits")
dr16data = dr16file[1].data

In [3]:
# MEK take only objects that have more than one observation
idx_repeat = np.where(dr16data["NSPEC"]>0)[0]
repeat_data = dr16data[idx_repeat]

In [4]:
# MEK take only objects below a z=0.8039 (SDSS spectrograph red limit with 5100 redshifted to edge, to keep H-Beta)
idx_z = np.where(repeat_data["Z"]<0.8039)[0]
repeat_z_data = repeat_data[idx_z]

In [5]:
# MEK create storage for plates, MJDs, and fibers
names = []
plates = []
mjds = []
fibers = []

# MEK go through each object
for data in repeat_z_data:
    # MEK wipe any information about the previous object
    name = []
    plate = []
    mjd = []
    fiber = []
    
    # MEK take primary data
    sdssname = data["SDSS_NAME"]
    name.append(sdssname)
    plate.append(data["PLATE"])
    mjd.append(data["MJD"])
    fiber.append(data["FIBERID"])
    
    # MEK take duplicate data, ignoring the filler -1s that bring each entry to a length of 74
    for num in data["PLATE_DUPLICATE"]:
        if num == -1:
            pass
        else:
            name.append(sdssname)
            plate.append(num)
    for num in data["MJD_DUPLICATE"]:
        if num == -1:
            pass
        else:
            mjd.append(num)
    for num in data["FIBERID_DUPLICATE"]:
        if num == -1:
            pass
        else:
            fiber.append(num)
            
    # MEK add data from one source to the growing list of all the sources        
    names.append(name)
    plates.append(plate)
    mjds.append(mjd)
    fibers.append(fiber)

print((names[9]))
print((plates[9]))
print((mjds[9]))
print((fibers[9]))

['000104.52+002753.7', '000104.52+002753.7']
[9403, 1091]
[58018, 52902]
[460, 521]


In [6]:
# MEK establish the link we will pull from
sdss_archive_str = "https://data.sdss.org/sas/dr16/sdss/spectro/redux/"

# MEK create lists to store the names of the spectra (nofits for making folders, fits for wget, plates for wget path)
all_files_nofits = []
all_files_fits = []
plates_for_path = []
names_for_path = []

# MEK for all of the plates, MJDs, and fibers that we have - put them together into the format that SDSS uses to name spectra
for name, plate, mjd, fiber in zip(names, plates, mjds, fibers):
    for i in range(0, len(plate)):
        names_for_path.append(name[i])
        filename_nofits = "spec-"+str(plate[i]).zfill(4)+"-"+str(mjd[i])+"-"+str(fiber[i]).zfill(4)
        filename_fits = "spec-"+str(plate[i]).zfill(4)+"-"+str(mjd[i])+"-"+str(fiber[i]).zfill(4)+".fits"
        all_files_nofits.append(filename_nofits)
        all_files_fits.append(filename_fits)
        plates_for_path.append(plate[i])
        #os.system("mkdir "+filename_nofits)

print(len(all_files_nofits))
print(len(all_files_fits))
print(len(plates_for_path))
print(len(names_for_path))

25359
25359
25359
25359


In [7]:
# MEK maybe not necessary - break up the fits that we'll wget into more bite sized pieces
fits_1 = all_files_fits[0:5000]
plates_1 = plates_for_path[0:5000]
fits_2 = all_files_fits[5000:10000]
plates_2 = plates_for_path[5000:10000]
fits_3 = all_files_fits[10000:15000]
plates_3 = plates_for_path[10000:15000]
fits_4 = all_files_fits[15000:20000]
plates_4 = plates_for_path[15000:20000]
fits_5 = all_files_fits[20000:]
plates_5 = plates_for_path[20000:]

In [8]:
'''
# MEK go into each directory
os.chdir(all_files_nofits[2])

# MEK try all available paths where the data might live
pathnames = ["26/spectra/lite/", "103/spectra/lite/", "104/spectra/lite/", "v5_13_0/spectra/lite/"]

# MEK wget for all paths (wget will keep going if it doesn't find the file you've requested)
for path in pathnames:
    os.system("wget "+sdss_archive_str+path+str(plates_for_path[2]).zfill(4)+"/"+all_files_fits[2])

# MEK go back to main directory  
os.chdir("..")
'''

'\n# MEK go into each directory\nos.chdir(all_files_nofits[2])\n\n# MEK try all available paths where the data might live\npathnames = ["26/spectra/lite/", "103/spectra/lite/", "104/spectra/lite/", "v5_13_0/spectra/lite/"]\n\n# MEK wget for all paths (wget will keep going if it doesn\'t find the file you\'ve requested)\nfor path in pathnames:\n    os.system("wget "+sdss_archive_str+path+str(plates_for_path[2]).zfill(4)+"/"+all_files_fits[2])\n\n# MEK go back to main directory  \nos.chdir("..")\n'

In [9]:
# MEK take only objects below a z=0.8039 (SDSS spectrograph red limit with 5100 redshifted to edge, to keep H-Beta)
idx_dr16z = np.where(dr16data["Z"]<0.8039)[0]
dr16z_data = dr16data[idx_dr16z]

# MEK create SkyCoords of all DR16Q
dr16_ra = dr16z_data["RA"]
dr16_dec = dr16z_data["DEC"]
dr16_pos = SkyCoord(dr16_ra, dr16_dec, unit=(u.deg, u.deg))

In [10]:
# MEK pull DR18 data
dr18file = fits.open("../../spAll-v6_0_4.fits")
dr18data = dr18file[1].data

In [11]:
# MEK take only objects below a z=0.8039 (SDSS spectrograph red limit with 5100 redshifted to edge, to keep H-Beta)
idx_dr18z = np.where(dr18data["Z"]<0.8039)[0]
dr18z_data = dr18data[idx_dr18z]

# MEK create SkyCoords of all DR18
dr18_ra = dr18z_data["PLUG_RA"]
dr18_dec = dr18z_data["PLUG_DEC"]
dr18_pos = SkyCoord(dr18_ra, dr18_dec, unit=(u.deg, u.deg))

In [12]:
# MEK compare where DR16 and DR18 sources are within 5 arcseconds of each other - these are repeat observations between the two
sep = 5/60/60
id1, id2, d2, d3 = dr18_pos.search_around_sky(dr16_pos, sep*u.deg)

In [13]:
# MEK grab only the positions that are in both catalogues
overlap16 = dr16_pos[id1]
overlap18 = dr18_pos[id2]
print(np.shape(id1))
print(np.shape(id2))

(414,)
(414,)


In [14]:
# MEK grab the DR16 names of each object
dr16names = list(repeat_z_data["SDSS_NAME"])
overlapnames = dr16z_data[id1]["SDSS_NAME"]

In [15]:
# MEK combine lists (ignore duplicate names!)
for name in overlapnames:
    if name in dr16names:
        pass
    else:
        dr16names.append(name)

In [16]:
# MEK look for repeats in DR18
idx_dr18z_repeat = np.where(dr18z_data["NSPECOBS"]>1)[0]
dr18z_repeat_data = dr18z_data[idx_dr18z_repeat]

In [17]:
ras = dr18z_repeat_data["PLUG_RA"]
decs = dr18z_repeat_data["PLUG_DEC"]
coords = SkyCoord(ras, decs, unit=(u.deg, u.deg))
hms = coords.ra.hms
dms = coords.dec.dms

# SDSS JHHMMSS.ss+DDMMSS.s

hmslist = []

for h, m, s in zip(hms[0], hms[1], hms[2]):
    h = str(round(h)).zfill(2)
    m = str(round(m)).zfill(2)
    s = f"{s:.2f}"
    s = s.zfill(5)
    hms_str = h+m+s
    hmslist.append(hms_str)
    
dmslist = []
for d, m, s in zip(dms[0], dms[1], dms[2]):
    if d<0:
        d = str(round(d)).zfill(3)
        m = str(-1*(round(m))).zfill(2)
        s = str(-1*(round(s, 1))).zfill(4)
    else:
        if d<1 and m<0 and s<0:
            if -2<m<2:
                d = "-"+str(round(d)).zfill(2)
                m = str(-1*(round(m))).zfill(2)
                s = str(-1*(round(s, 1))).zfill(4)
                #print("first", d, m ,s)
            else:
                d = "-"+str(round(d)).zfill(2)
                m = str(-1*(round(m))).zfill(2)
                s = str(-1*(round(s, 1))).zfill(4)
        else:
            d = "+"+str(round(d)).zfill(2)
            m = str(round(m)).zfill(2)
            s = str(round(s, 1)).zfill(4)
    dms_str = d+m+s
    dmslist.append(dms_str)

j2000list = []
for ra, dec in zip(hmslist, dmslist):
    j2000 = ra+dec
    j2000list.append(j2000)
    #print(j2000)

i = 0
troublesomeindices = []
while i < len(j2000list):
    if j2000list[i][10:15] == "0000-":
        troublesomeindices.append(i)
    i += 1
    
troublesomeindices = np.array(troublesomeindices)
bad = np.array(j2000list)[troublesomeindices]

In [4]:
ipl3file = fits.open("../../spAll-lite-v6_1_3.fits")
ipl3data = ipl3file[1].data
ipl3_hdr = ipl3file[1].header

In [6]:
idx_ipl3z = np.where(ipl3data["Z"]<0.8039)[0]
ipl3z_data = ipl3data[idx_ipl3z]
ipl3_ra = ipl3z_data["RACAT"]
ipl3_dec = ipl3z_data["DECCAT"]
ipl3_pos = SkyCoord(ipl3_ra, ipl3_dec, unit=(u.deg, u.deg))

ipl3epoch = ipl3z_data["COORD_EPOCH"]
print(ipl3epoch)

[2000. 2000. 2000. ...   nan   nan   nan]


In [20]:
# MEK compare where DR16 and DR18 sources are within 5 arcseconds of each other - these are repeat observations between the two
sep = 5/60/60
id1, id2, d2, d3 = ipl3_pos.search_around_sky(dr16_pos, sep*u.deg)
# MEK grab only the positions that are in both catalogues
overlap16 = dr16_pos[id1]
overlapipl = ipl3_pos[id2]
print(np.shape(id1))
print(np.shape(id2))

(34340,)
(34340,)


In [7]:
flags = TargetingFlags(ipl3z_data["SDSS5_TARGET_FLAGS"])

In [22]:
'''for label, count in flags.count(skip_empty=True).items():
    print(f"Label: {label}, Count: {count}")'''

'for label, count in flags.count(skip_empty=True).items():\n    print(f"Label: {label}, Count: {count}")'

In [23]:
'''for mapper, count in flags.count_by_attribute("mapper").items():
    print(f"{mapper}: {count:,}")'''

'for mapper, count in flags.count_by_attribute("mapper").items():\n    print(f"{mapper}: {count:,}")'

In [24]:
'''for program, count in flags.count_by_attribute("program").items():
    print(f"{program}: {count:,}")'''

'for program, count in flags.count_by_attribute("program").items():\n    print(f"{program}: {count:,}")'

In [27]:
rm_idx = flags.in_program("bhm_rm")
rm = ipl3z_data[rm_idx]

aqmes_idx = flags.in_program("bhm_aqmes")
aqmes = ipl3z_data[aqmes_idx]

In [26]:
rm_ra = rm["RACAT"]
rm_dec = rm["DECCAT"]

aqmes_ra = aqmes["RACAT"]
aqmes_dec = aqmes["DECCAT"]

ipl3_q_ra = np.concatenate((rm_ra, aqmes_ra), axis=0)
ipl3_q_dec = np.concatenate((rm_dec, aqmes_dec), axis=0)

In [27]:
ipl3_q_pos = SkyCoord(ipl3_q_ra, ipl3_q_dec, unit=(u.deg, u.deg))

In [28]:
# MEK compare where DR16 and DR18 sources are within 5 arcseconds of each other - these are repeat observations between the two
sep = 5/60/60
id1, id2, d2, d3 = ipl3_q_pos.search_around_sky(dr16_pos, sep*u.deg)
# MEK grab only the positions that are in both catalogues
overlap16 = dr16_pos[id1]
overlapiplq = ipl3_q_pos[id2]
print(np.shape(id1))
print(np.shape(id2))

missing_dr16q = len(ipl3_q_pos)-len(id2)
print(missing_dr16q)
print(np.shape(ipl3_q_pos))

print("overlap", overlapiplq[0:10])
print(id1[0:10])
print(id2[0:10])

rm_aqmes = np.concatenate((rm, aqmes), axis=0)
print(rm_aqmes[id2][0:10]["MJD"])

(16113,)
(16113,)
21023
(37136,)
overlap <SkyCoord (ICRS): (ra, dec) in deg
    [(0.04984221,  0.0403696 ), (0.04984221,  0.0403696 ),
     (0.04984224,  0.04036958), (0.04984224,  0.04036958),
     (0.04984224,  0.04036958), (0.06847029, -0.3092766 ),
     (0.06847029, -0.3092766 ), (0.06847035, -0.30927659),
     (0.06847035, -0.30927659), (0.06847035, -0.30927659)]>
[12 12 12 12 12 14 14 14 14 14]
[27925 27970 34365 35047 35069 27914 27959 34351 35034 35056]
[59189 59190 59844 59897 59898 59189 59190 59844 59897 59898]


In [29]:
no_match = []
for i in range(0, len(ipl3_q_pos)):
    if i not in id2:
        no_match.append(i)
        
no_matches = ipl3_q_pos[no_match]
print(len(no_matches))

21026


In [30]:
'''
names16 = repeat_z_data["SDSS_NAME"]
print(np.shape(names16))

namesipl = dr16z_data[id1]["SDSS_NAME"]
print(np.shape(namesipl))

all_names = np.concatenate((names16, namesipl), axis=0)
print(np.shape(all_names))

all_names = np.unique(all_names)
print(np.shape(all_names))
print(all_names)

os.system("pwd")

print(np.shape(names_for_path))
print(np.shape(plates_for_path))
print(np.shape(all_files_fits))

for name, plate, file in zip(names_for_path, plates_for_path, all_files_fits):
    #os.system("mkdir "+name)
    # MEK go into each directory
    os.chdir(name)
    os.system("pwd")
    
    # MEK try all available paths where the data might live
    pathnames = ["26/spectra/lite/", "103/spectra/lite/", "104/spectra/lite/", "v5_13_0/spectra/lite/"]
    
    # MEK wget for all paths (wget will keep going if it doesn't find the file you've requested)
    for path in pathnames:
        command="wget "+sdss_archive_str+path+str(plate).zfill(4)+"/"+file
        #print(command)
        #os.system(command)
    
    # MEK go back to main directory  
    os.chdir("..")
'''

'\nnames16 = repeat_z_data["SDSS_NAME"]\nprint(np.shape(names16))\n\nnamesipl = dr16z_data[id1]["SDSS_NAME"]\nprint(np.shape(namesipl))\n\nall_names = np.concatenate((names16, namesipl), axis=0)\nprint(np.shape(all_names))\n\nall_names = np.unique(all_names)\nprint(np.shape(all_names))\nprint(all_names)\n\nos.system("pwd")\n\nprint(np.shape(names_for_path))\nprint(np.shape(plates_for_path))\nprint(np.shape(all_files_fits))\n\nfor name, plate, file in zip(names_for_path, plates_for_path, all_files_fits):\n    #os.system("mkdir "+name)\n    # MEK go into each directory\n    os.chdir(name)\n    os.system("pwd")\n    \n    # MEK try all available paths where the data might live\n    pathnames = ["26/spectra/lite/", "103/spectra/lite/", "104/spectra/lite/", "v5_13_0/spectra/lite/"]\n    \n    # MEK wget for all paths (wget will keep going if it doesn\'t find the file you\'ve requested)\n    for path in pathnames:\n        command="wget "+sdss_archive_str+path+str(plate).zfill(4)+"/"+file\n 

In [31]:
sdss_matches = dr16z_data[id1]
rm_aqmes_matches = rm_aqmes[id2]
ipl3_overlap_sdss_names = sdss_matches["SDSS_NAME"]
print(len(ipl3_overlap_sdss_names))
os.system("pwd")

first50names = ipl3_overlap_sdss_names[0:50]
print(first50names)

'''
for name in ipl3_overlap_sdss_names:
    os.system("mkdir "+name)
'''

16113
/home/kaldorme/research/code/repeat_obs
['000011.96+000225.2' '000011.96+000225.2' '000011.96+000225.2'
 '000011.96+000225.2' '000011.96+000225.2' '000016.43-001833.4'
 '000016.43-001833.4' '000016.43-001833.4' '000016.43-001833.4'
 '000016.43-001833.4' '000109.12-004121.8' '000109.12-004121.8'
 '000109.12-004121.8' '000109.12-004121.8' '000109.12-004121.8'
 '000109.12-004121.8' '000111.19-002011.5' '000111.19-002011.5'
 '000111.19-002011.5' '000111.19-002011.5' '000111.19-002011.5'
 '000111.19-002011.5' '000114.75+264127.5' '000114.75+264127.5'
 '000114.75+264127.5' '000114.75+264127.5' '000116.48+263257.9'
 '000116.48+263257.9' '000116.48+263257.9' '000116.48+263257.9'
 '000134.71-003103.6' '000134.71-003103.6' '000134.71-003103.6'
 '000134.71-003103.6' '000134.71-003103.6' '000134.71-003103.6'
 '000140.70+260425.5' '000140.70+260425.5' '000140.70+260425.5'
 '000140.70+260425.5' '000208.83-001742.7' '000208.83-001742.7'
 '000208.83-001742.7' '000208.83-001742.7' '000208.83-0017

'\nfor name in ipl3_overlap_sdss_names:\n    os.system("mkdir "+name)\n'

In [32]:
specfile = rm_aqmes_matches["SPEC_FILE"]
specfile = [file.decode("UTF-8") for file in specfile]

dir1 = [name[5:11] for name in specfile]
print(dir1[0:5])

dir2 = [name[12:17] for name in specfile]
print(dir2[0:5])

print(specfile[0:5])

print(np.shape(dir1))
print(np.shape(dir2))
print(np.shape(specfile))
print(np.shape(ipl3_overlap_sdss_names))

['015035', '015035', '104634', '104634', '104634']
['59189', '59190', '59844', '59897', '59898']
['spec-015035-59189-4399101101.fits', 'spec-015035-59190-4399101101.fits', 'spec-104634-59844-27021597957478658.fits', 'spec-104634-59897-27021597957478658.fits', 'spec-104634-59898-27021597957478658.fits']
(16113,)
(16113,)
(16113,)
(16113,)


In [33]:
ipl3_path = "https://data.sdss5.org/sas/ipl-3/spectro/boss/redux/v6_1_3/spectra/lite/"
usr = "sdss5"
pss = "panoPtic-5"

'''
for name, dir1_str, dir2_str, specfile_str in zip(ipl3_overlap_sdss_names, dir1, dir2, specfile):
    # MEK go into each directory
    os.chdir(name)
    os.system("pwd")
    
    # MEK wget for all paths (wget will keep going if it doesn't find the file you've requested)
    command="wget --user "+usr+" --password "+pss+" "+ipl3_path+dir1_str+"/"+dir2_str+"/"+specfile_str
    print(command)
    os.system(command)
    
    # MEK go back to main directory  
    os.chdir("..")
'''

'\nfor name, dir1_str, dir2_str, specfile_str in zip(ipl3_overlap_sdss_names, dir1, dir2, specfile):\n    # MEK go into each directory\n    os.chdir(name)\n    os.system("pwd")\n    \n    # MEK wget for all paths (wget will keep going if it doesn\'t find the file you\'ve requested)\n    command="wget --user "+usr+" --password "+pss+" "+ipl3_path+dir1_str+"/"+dir2_str+"/"+specfile_str\n    print(command)\n    os.system(command)\n    \n    # MEK go back to main directory  \n    os.chdir("..")\n'

In [34]:
'''
spec-015002-59230-6358921707
59229
59228
'''

'\nspec-015002-59230-6358921707\n59229\n59228\n'

In [70]:
# previous run stopped at index 12718 (completed the run at index 12718, but index 12719 did not start)
print(np.where(specfile == 'spec-015171-59264-4350967940.fits'))
idx = np.where(ipl3_overlap_sdss_names == '141625.70+535438.5')[0]
print(idx)
print(ipl3_overlap_sdss_names[12718])
print(specfile[12718])
print(dir1[12718])
print(dir2[12718])

(array([], dtype=int64),)
[12702 12703 12704 12705 12706 12707 12708 12709 12710 12711 12712 12713
 12714 12715 12716 12717 12718 12719 12720 12721 12722 12723 12724 12725
 12726 12727 12728 12729 12730 12731 12732 12733 12734 12735 12736 12737
 12738 12739 12740 12741 12742 12743 12744 12745 12746 12747 12748 12749
 12750 12751 12752 12753 12754 12755 12756 12757 12758 12759 12760 12761
 12762 12763]
141625.70+535438.5
spec-015171-59264-4350967940.fits
015171
59264


/tmp/ipykernel_4272/545518558.py:2: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  print(np.where(specfile == 'spec-015171-59264-4350967940.fits'))


In [77]:
ipl3_overlap_sdss_names_end = ipl3_overlap_sdss_names[12719:]
dir1_end = dir1[12719:]
dir2_end = dir2[12719:]
specfile_end = specfile[12719:]

print(ipl3_overlap_sdss_names_end[0])
print(dir1_end[0])
print(dir2_end[0])
print(specfile_end[0])

print(np.shape(ipl3_overlap_sdss_names_end))
print(np.shape(dir1_end))
print(np.shape(dir2_end))
print(np.shape(specfile_end))

141625.70+535438.5
015171
59265
spec-015171-59265-4350967940.fits
(3394,)
(3394,)
(3394,)
(3394,)


In [88]:
os.system("pwd")

'''
for name, dir1_str, dir2_str, specfile_str in zip(ipl3_overlap_sdss_names_end, dir1_end, dir2_end, specfile_end):
    # MEK go into each directory
    os.chdir(name)
    os.system("pwd")
    
    # MEK wget for all paths (wget will keep going if it doesn't find the file you've requested)
    command="wget --user "+usr+" --password "+pss+" "+ipl3_path+dir1_str+"/"+dir2_str+"/"+specfile_str
    print(command)
    os.system(command)
    
    # MEK go back to main directory  
    os.chdir("..")
'''

/home/kaldorme/research/code/repeat_obs


'\nfor name, dir1_str, dir2_str, specfile_str in zip(ipl3_overlap_sdss_names_end, dir1_end, dir2_end, specfile_end):\n    # MEK go into each directory\n    os.chdir(name)\n    os.system("pwd")\n    \n    # MEK wget for all paths (wget will keep going if it doesn\'t find the file you\'ve requested)\n    command="wget --user "+usr+" --password "+pss+" "+ipl3_path+dir1_str+"/"+dir2_str+"/"+specfile_str\n    print(command)\n    os.system(command)\n    \n    # MEK go back to main directory  \n    os.chdir("..")\n'

In [87]:
nobs_ipl3 = ipl3z_data["NSPECOBS"]
multiobs_ipl3 = np.where(nobs_ipl3 > 1)[0]
print(np.shape(ipl3z_data))
print(np.shape(multiobs_ipl3))

(1509804,)
(232142,)
